# EE4375: Sixth Lab Session Extensions: Galerkin Finite Element Method for the Heat and Wave Equation on the Unit Square 

Solves the Heat equation and the wave on the unit square domain $\Omega=(0,1)^2$ supplied with the homogeneous Dirichlet boundary condition $u = 0$ on $\Gamma$. Linear finite elements on triangular elements are used for the spatial discretization. DifferentialEquations.jl is used for the time integration. 

## Import Packages

In [1]:
import Gmsh: gmsh 
using GR 
using LinearAlgebra
using SparseArrays 
using DifferentialEquations
using Plots 

## Section 1: Spatial discretization (take from 6th lab session)

## Section 2: Solving the Scalar Diffusion Equation in Time Domain 

### Defining the Problem 

In [9]:
#..define the right-hand side of the ordinary differential equation of the equation of motion 
function scalarDiffusionEquation!(du,u,p,t)
    du .= M \ (sin(2*π*50*t)*f - A*u)
end

#..set initial position and velocity
u0 = fill(1.,nnodes)
                                    
#..set time begin and end forward
t0 = 0.0
tf = 2.0
tspan = (t0,tf)

#..define ODE problem to be solved  
prob_scalarDiffusion = ODEProblem(scalarDiffusionEquation!, u0, tspan)

#..solve ODE problem 
sol = DifferentialEquations.solve(prob_scalarDiffusion);

### Plotting the computed solution 
Requires code optimization (unclear why and how). 

In [ ]:
dt = 2
tvec = Vector(t0:dt:tf)

Plots.plot(sol)
for i in 2:nnodes-1
    Plots.plot!(sol,vars=i)
end
p1 = Plots.plot!(sol)
title!("Displacement")

#..plot solution of velocity and position as function of time  
Plots.plot(p1)

In [ ]:
anim = @animate for i in t0:tf
    Plots.surface(xnode,ynode,sol(i/10),st=:surface)
    zlims!(-.2, 1.5)
end
gif(anim, "my-animation.gif", fps = 1)

## Section 3: Solving a Wave Equation in Time Domain 

In [ ]:
# set damping matrix
C = .005*M 

#..define the right-hand side of the ordinary differential equation of the equation of motion 
function scalarWaveEquation!(ddu,du,u,p,t)
    ddu .= M \ (f - A*u - C*du)
end

#..set initial position and velocity
u0 = fill(0.,nnodes)

v0 = zeros(nnodes)
# v0[4] = 1
                                    
#..set time begin and end forward
t0 = 0.0
tf = 20.0
tspan = (t0,tf)           

#..define ODE problem to be solved  
prob = SecondOrderODEProblem(scalarWaveEquation!,v0,u0,tspan)

#..solve ODE problem 
sol = DifferentialEquations.solve(prob);

### Post-plotting the Solution using time-traces  

In [ ]:
dt = 1
tvec = Vector(t0:dt:tf)

#..velocity and position have vars=(1:N) and vars=(N+1,2N), respectively. 
Plots.plot(sol,vars=1)
for i in 2:nnodes-1
    Plots.plot!(sol,vars=i)
end
p1 = Plots.plot!(sol,vars=nnodes)
title!("Velocity")

plot(sol,vars=nnodes+1)
for i in nnodes+2:2*nnodes-1
    Plots.plot!(sol,vars=i)
end
p2 = Plots.plot!(sol,vars=2*nnodes)
title!("Displacement")

#..plot solution of velocity and position as function of time  
Plots.plot(p1,p2,layout=(2,1))

### Post-plotting using animation 

In [ ]:
anim = @animate for i in t0:tf
    Plots.surface(xnode,ynode,sol(i/10)[nnodes+1:2*nnodes],st=:surface)
    zlims!(-.2, .4)
end
gif(anim, "my-animation.gif", fps = 15)